In [1]:
import torch
from torch import nn

In [10]:
def corr2d(X, K):
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1,X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i + h, j:j + w] * K).sum()
    return Y

X = torch.tensor([[0., 1., 2.],[3., 4., 5.],[6., 7., 8.]])
K = torch.tensor([[0., 1.],[2., 3.]])

corr2d(X, K)

tensor([[19., 25.],
        [37., 43.]])

In [5]:
class Conv2d(nn.Module):
    def __init__(self,kernal_size):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))

    def forward(self,X):
        return corr2d(X,self.weight) + self.bias

In [18]:
X = torch.ones((6,8))
X [: , 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [19]:
K = torch.tensor([[1., -1.]])
Y = corr2d(X, K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

In [13]:
corr2d(X.t(), K)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

In [27]:
conv2d = nn.Conv2d(1,1,kernel_size = (1,2), bias = False)

# (批量大小，通道，高度，宽度)
X = X.reshape((1,1,6,8))
Y = Y.reshape((1,1,6,7))

lr = 3e-2

print((conv2d(X) - Y) ** 2)

for i in range(10):
    Y_hat = conv2d(X)
    l = (Y_hat - Y) ** 2
    conv2d.zero_grad()
    l.sum().backward()
    # 迭代卷积核
    conv2d.weight.data[:] -= lr * conv2d.weight.grad
    if (i + 1) % 2 == 0:
        print(f'epoch {i + 1}, loss {l.sum():.3f}')

tensor([[[[0.0656, 0.8147, 0.0000, 0.0000, 0.0000, 0.4179, 0.0656],
          [0.0656, 0.8147, 0.0000, 0.0000, 0.0000, 0.4179, 0.0656],
          [0.0656, 0.8147, 0.0000, 0.0000, 0.0000, 0.4179, 0.0656],
          [0.0656, 0.8147, 0.0000, 0.0000, 0.0000, 0.4179, 0.0656],
          [0.0656, 0.8147, 0.0000, 0.0000, 0.0000, 0.4179, 0.0656],
          [0.0656, 0.8147, 0.0000, 0.0000, 0.0000, 0.4179, 0.0656]]]],
       grad_fn=<PowBackward0>)
epoch 2, loss 3.578
epoch 4, loss 0.753
epoch 6, loss 0.189
epoch 8, loss 0.057
epoch 10, loss 0.020


In [26]:
conv2d.weight.data.reshape((1, 2))

tensor([[ 0.9461, -1.0286]])